<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/Copia_de_2_obtencion_textos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=1pWL34MWc1rS0peIekOBe8GGBIEUFPyrG" width="100%">

# **Obtención de Textos desde _Python_**
---

En algunos casos no podemos cargar información textual directamente desde un archivo plano o desde un formato estándar como `json`. En estos casos debemos realizar un proceso comúnmente conocido como **parsing** para extraer textos desde distintas fuentes. En este taller guiado veremos las fuentes de textos más típicas y cómo podemos extraer información de ellas, en especial:

* Extracción de texto en archivos `pdf`.
* Extracción de texto en imágenes con `ocr`.
* Extracción de texto desde la web.

## **1. Extracción de Texto en Archivos PDF**
---

El formato de archivos `pdf` (Portable Document Format) fue creado por *Adobe* en 1992 para presentar documentos con contenido basado en texto e imágenes. Este tipo de archivos se caracterizan por almacenar la información en un formato generalmente binario de forma compacta y eficiente.

### **1.1. Instalación**
---

Para el ejercicio de obtención de información textual a partir de archivos `pdf` haremos uso de la herramienta [MuPDF](https://mupdf.com/). Se trata de una herramienta simple que permite extraer información de archivos en formato `pdf` con facilidad.

<img src="https://drive.google.com/uc?export=view&id=1tuK7RZPWBXVG1n7VswLGcY_xUkJ2yawb" width="30%">

En específico, usaremos la librería `pymupdf` para extraer información de archivos `pdf` desde _Python_. Adicionalmente, instalamos la librería `Pillow` para manejo de imágenes desde _Python_:

In [ ]:
!pip install Pillow==9.4.0
!pip install pymupdf

> **NOTA**: Después de la instalación, es posible que Google Colaboratory le pida reiniciar el entorno de ejecución (*RUNTIME*). Puede hacerlo, haciendo clic en el botón `RESTART RUNTIME` antes de continuar.
 <img src="https://drive.google.com/uc?export=view&id=1x9WLH8bLR5i6yV-NoOheOmlVX2C9l07F" width="80%">

La librería `pymupdf` se importa mediante el paquete `fitz`, como mostramos a continuación. También importamos otras librerías generales como `os` y `matplotlib` para visualización y validación del sistema de archivos.

In [ ]:
import fitz
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

### **1.2. Carga de Datos**
---

En este caso, veremos un ejemplo de carga, extracción y modificación de un archivo `pdf` que corresponde a un documento de [tesis de maestría](https://repositorio.unal.edu.co/handle/unal/78687) disponible en el repositorio digital de la Universidad Nacional de Colombia.

Descargamos el archivo y lo guardamos como `tesis.pdf`:

In [ ]:
!wget 'https://drive.google.com/uc?export=view&id=18Lef8IqqSCULq_MuUDn9cR_o1M4MXnGE' -O 'tesis.pdf'

Podemos, corroborar que el archivo fue descargado correctamente y existe dentro de nuestro entorno en Google Colaboratory:

In [ ]:
print(os.path.exists("tesis.pdf"))

También podemos corroborar el tamaño de este archivo:

In [ ]:
nbytes = os.stat("tesis.pdf").st_size
megabytes = nbytes / (1024 ** 2)
print(f"El archivo tiene {megabytes:.2f} MB")

Veamos cómo obtener datos de este archivo con la librería `fitz`. Para comenzar a manipular el archivo `pdf` debemos leerlo desde la librería; para ello, hacemos uso de la función `fitz.open`, especificando la ruta del archivo como se muestra en la siguiente celda:

In [ ]:
pdf = fitz.open("tesis.pdf")

Podemos validar que se cargó un objeto de tipo `Document` de `fitz`

In [ ]:
print(type(pdf))

### **1.3. Extracción de Metadatos**
---

Sobre este documento podemos comenzar a extraer metadatos generales como:

* `author`: nombre del autor del documento.
* `creationDate`: fecha de creación del documento.
* `creator`: nombre del programa con el que fue creado el documento.
* `encryption`: especifica si el archivo tiene algún mecanismo de encriptado.
* `format`: tipo y versión del archivo cargado.
* `keywords`: palabras clave asociadas al documento.
* `modDate`: última fecha de modificación del documento.
* `producer`: herramienta `pdf` usada para generar el archivo.
* `subject`: resumen del documento.
* `title`: título del documento.

Para extraer estos valores, usamos el atributo `metadata` de un `Document` en `fitz` como se muestra a continuación:

In [ ]:
print(pdf.metadata)

También es posible extraer el número de páginas que tiene el documento, esto lo conseguimos al extraer la longitud del documento con la función `len`:

In [ ]:
print(len(pdf))

### **1.4. Extracción de Páginas**
---

Para extraer información de un documento dentro de `fitz` debemos extraer una página, esto lo hacemos con el método `load_page` de un `Document`:

In [ ]:
page0 = pdf.load_page(0)

En este caso, obtuvimos la página 0 del documento. Podemos validar su tipo:

In [ ]:
print(type(page0))

En especial, estaremos trabajando sobre el objeto `Page` para la extracción de información.

Veamos un pantallazo de lo que contiene esta página con el método `get_pixmap` para obtener una imagen de la misma:

In [ ]:
img = page0.get_pixmap()

Podemos validar su tipo:

In [ ]:
print(type(img))

El objeto `Pixmap` puede exportarse como una imagen con el método `save` y especificando una ruta:

In [ ]:
img.save("page0.png")

Podemos validar que la imagen se creó:

In [ ]:
print(os.path.exists("page0.png"))

Veamos la imagen como una visualización de `matplotlib`. Primero, cargamos la imagen:

In [ ]:
img = mpimg.imread("page0.png")

Ahora, podemos generar una gráfica de la misma:

In [ ]:
fig, ax = plt.subplots(figsize=(7, 10))
ax.imshow(img)
ax.axis("off")

### **1.5. Extracción de Información**
---

A partir de una página podemos extraer su contenido textual con el método `get_text`, este recibe una opción para el parsing dentro de las siguientes:

* `text`: obtiene el texto crudo con saltos de línea, sin detalles como formatos e imágenes.
* `blocks`: obtiene los párrafos.
* `words`: obtiene una lista de palabras.
* `html`: genera una versión en formato html de la página con las imágenes embebidas.
* `json`: guarda la información en formato JSON.
* `xhtml`: similar al formato HTML.
* `xml`: guarda la información textual en formato XML sin imágenes.

Veamos cómo extraer el texto de la página que teníamos cargada:

In [ ]:
text = page0.get_text("text")
print(text)

También podemos extraer el texto de otra página:

In [ ]:
text = (
        pdf
        .load_page(9)
       .get_text("text")
        )
print(text)

Si deseamos cargar el texto del documento completo, podemos iterar sobre todas las páginas del mismo.

Veamos un ejemplo:

In [ ]:
texts = [page.get_text("text") for page in pdf]

Podemos corroborar que el total de las páginas fuese cargado:

In [ ]:
print(len(texts))

Veamos el texto de las primeras 10 páginas del documento:

In [ ]:
for page in texts[:10]:
    print(page)
    print("---")

## **2. Extracción de Texto en Imágenes con OCR**
---

En muchas ocasiones la información textual se encuentra contenida dentro de imágenes debido a la naturaleza de su adquisición, por ejemplo, pantallazos, imágenes escaneadas, fotografías, entre otras. Ante esto, existe una metodología para la extracción de los textos conocida como reconocimiento óptico de caracteres: **Optical Character Recognition** (OCR).

Se trata de un proceso en el que se identifican regiones dentro de una imagen que contienen información textual, como se muestra en la siguiente figura:

<img src="https://drive.google.com/uc?export=view&id=1oNIJ70j29ruh3mtnfMcidthtdINCTavI" width="65%">

### **2.1. Instalación**
---

Para hacer OCR desde _Python_ podemos usar la herramienta de software libre [Tesseract](https://tesseract-ocr.github.io/), la cual es financiada por Google desde el año 2005 y es considerado como uno de los motores de OCR de código abierto más precisos disponibles.

<img src="https://drive.google.com/uc?export=view&id=14pllP0PotBSFg04WU1GAQYgLPliW9Kv-" width="50%">

Veamos cómo instalar esta herramienta:

In [ ]:
!apt install tesseract-ocr

Si deseamos realizar OCR en español, debemos instalar el paquete de lenguaje para español con el siguiente comando:

In [ ]:
!apt install tesseract-ocr-spa

De forma general, puede instalar paquetes de alrededor de 124 idiomas distintos. Puede consultar el listado completo y el código de lenguaje en [este enlace](https://tesseract-ocr.github.io/tessdoc/Data-Files-in-different-versions.html).

Para instalar otro paquete de idioma puede modificar el comando:

```sh
!apt install tesseract-ocr-<lang>
```

Reemplazando `<lang>` por un código de idioma. Por ejemplo, para instalar OCR para el francés (código de lenguaje `fra`):

In [ ]:
!apt install tesseract-ocr-fra

Adicionalmente, podemos realizar OCR desde _Python_ con la librería `pytesseract`, la cual da una interacción con la herramienta `tesseract`, veamos su instalación:

In [ ]:
!pip install pytesseract

Con esto, podemos proceder a importar y utilizar `pytesseract` para OCR desde _Python_:

In [ ]:
import pytesseract

El proceso de OCR con esta librería es sencillo y consta de dos pasos:

1. Leer la imagen como un arreglo de `numpy`.
2. Extraer el texto de la imagen.

### **2.2. Carga de Datos**
---

Veamos un ejemplo con una imagen con el poema **Madrigal** de **José Asunción Silva**, primero descargamos la imagen:

In [ ]:
!wget 'https://drive.google.com/uc?export=view&id=1FY4OuPC2sdT1wnmPJsEjJ9qb21kxClPf' -O poema.png

Ahora procedemos a cargar la imagen como un arreglo de `numpy`:

In [ ]:
img = mpimg.imread("poema.png")

Podemos validar que efectivamente es un arreglo de `numpy`:

In [ ]:
print(type(img))

Así mismo, podemos ver el tamaño de esta imagen con el atributo `shape`:

In [ ]:
print(img.shape)

Como podemos ver, se trata de una imagen de dimensiones `(588, 813)` con 4 canales de color (rojo, verde, azul y transparencia).

Podemos visualizar la imagen con `matplotlib`:

In [ ]:
fig, ax = plt.subplots()
ax.imshow(img)
ax.axis("off");

### **2.3. Extracción de Información**
---

Para extraer el texto de esta imagen, usamos la función `image_to_string` de `pytesseract`. Esta función toma como argumento el arreglo de `numpy` de la imagen y el código de lenguaje `lang` sobre el cual se realizará OCR.

> **Nota**: `pytesseract` espera que la imagen esté codificada con el tipo `uint8`, por lo cual debemos hacer la conversión:

In [ ]:
img = (img * 255).astype("uint8")

Finalmente, realizamos la extracción del texto:

In [ ]:
text = pytesseract.image_to_string(img, lang="spa")
print(text)

## **3. Extracción de Texto Desde la Web**
---

Una gran cantidad de información textual se puede encontrar en contenidos de páginas web. El proceso de extracción de esta información se conoce como **Web Scraping** e involucra estrategias para el análisis de archivos en formato **HTML** (sitios estáticos) y la interactividad con páginas web (sitios dinámicos).

<img src="https://drive.google.com/uc?export=view&id=1EUUEi71gYhwO-GMfYXrH1t7dNTs5g8lI" width="50%">

### **3.1. Sitios Estáticos**
---

Gran parte del proceso de extracción de información de la web consiste en el análisis de documentos **HTML**. Esto es especialmente importante cuando hablamos de sitios web estáticos, los cuales se caracterizan por tener un contenido que se encuentra mayoritariamente mediante un archivo **HTML** y no hay mayor interactividad ni generación programática de contenido con `JavaScript`.

Desde _Python_ se puede automatizar la extracción de información de sitios de web estáticos con librerías como `beautifulsoup` y `requests`.

#### **3.1.1. Instalación**
---

La librería `requests` es muy conocida en _Python_ ya que nos da una forma fácil de utilizar protocolos de comunicación `http` y `https` desde _Python_.

<img src="https://drive.google.com/uc?export=view&id=1llgHPNuUR-m_aRtkLdYahma7KPdPeBub" width="20%">

Veamos cómo instalar `requests`:

In [ ]:
!pip install requests

Para importar la librería:

In [ ]:
import requests

Adicionalmente, para la extracción de información desde contenido web usaremos una librería llamada `beautifulsoup`. Se trata de un analizador y **parser** de archivos HTML y XML que es muy popular para web scraping desde _Python_.

<img src="https://drive.google.com/uc?export=view&id=1J5_ftObS6_5baMH2p82KFTC7dq9ZstOz" width="80%">

Veamos cómo instalar `beautifulsoup`

In [ ]:
!pip install beautifulsoup4

Para importar la librería:

In [ ]:
import bs4

#### **3.1.2. ¿Qué es HTML?**
---

El formato HTML toma su nombre de HyperText Markup Language, se trata del lenguaje estándar de marcado para páginas web, con el que típicamente se describe el contenido de un sitio y es uno de los pilares para la creación de páginas web.

El formato HTML estandariza el contenido que puede llegar a tener un sitio web por medio de distintos tipos de etiquetas, las cuales se representan de la siguiente forma

```html
<etiqueta>Contenido</etiqueta>
```

Un ejemplo muy sencillo de archivo `html` se muestra a continuación:

```html
<!DOCTYPE html>
<html>
<head>
    <title>Procesamiento y Entendimiento de Lenguaje Natural</title>
</head>
<body>
    <h1>Objetivo</h1>
    <p>La meta de este módulo es abordar los temas y conceptos fundamentales en el procesamiento de lenguaje natural (NLP, por sus siglas en inglés) por medios computacionales y con énfasis en textos escritos. Se estudiarán una amplia gama de técnicas para la lingüística computacional, el análisis estadístico del lenguaje y la minería de textos, aplicadas a problemas como: análisis sintáctico, extracción de información, clasificación y agrupamiento de texto, análisis de sentimientos, modelos de tópicos, entre otros. Todo ello, mediante el uso del lenguaje de programación Python y sus librerías especializadas para NLP.</p>
</body>
</html>
```

Esto nos muestra los elementos base que componen un documento **HTML**:

* `<!DOCTIPE html>`: una declaración que define que estamos trabajando con la versión **HTML5**.
* `<html>`: se trata de una etiqueta que acota donde inicia y termina el documento.
* `<head>`: contiene metadatos de la página.
* `<title>`: título de la página.
* `<body>`: se le conoce como el cuerpo del documento y contiene la información del mismo.

Los otros elementos, hacen parte de los distintos contenedores que pueden ser usados para visualizar el contenido de la página. Los tipos más comunes de contenedores son:

* `<p>`: párrafo de texto.
* `<h#>`: define un encabezado, por ejemplo `<h1>` define un título de primer nivel, mientras que `<h2>` describe un subtítulo.
* `<a>`: contiene un hipervínculo.
* `<ul>`: lista no ordenada de elementos.
* `<ol>`: lista ordenada de elementos (generalmente enumerados).
* `<li>`: elemento de una lista.
* `<table>`: define una tabla.
* `<th>`: encabezados de columnas en una tabla.
* `<tr>`: fila dentro de una tabla.
* `<td>`: celda de una tabla.
* `<img>`: imagen.
* `<video>`: video.
* `<div>`: bloque de contenido o sección de la página.
* `<section>`: bloque de contenido semántico de la página (generalmente asociado a tablas de contenido).

Puede consultar otras características y elementos de **HTML** en [este enlace](https://www.w3schools.com/html/default.asp).

#### **3.1.3. Carga de Datos**
---

Veamos un ejemplo práctico sobre una página estática, en específico usaremos la biografía de [Gabriel García Márquez en Wikipedia](https://es.wikipedia.org/wiki/Gabriel_Garc%C3%ADa_M%C3%A1rquez).

Para descargar el contenido de esta página usaremos la función `requests.get` a partir de la url de la página:

In [ ]:
url = "https://es.wikipedia.org/wiki/Gabriel_Garc%C3%ADa_M%C3%A1rquez"
r = requests.get(url)
print(type(r))

El paquete `requests` genera un objeto de tipo `Response`, generalmente podemos validar si la consulta fue satisfactoria si el valor de `status_code` es de 200.

In [ ]:
print(r.status_code)

> En algunas oportunidades puede tener errores al momento de acceder a una página web, los distintos tipos de códigos de estado de **HTTP** los puede consultar en [este enlace](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status).

Para obtener el contenido de la página, usamos el atributo `text` del objeto `Response` como se muestra a continuación:

In [ ]:
text = r.text

Validamos el tipo del texto obtenido:

In [ ]:
print(type(text))

Imprimimos los primeros 500 caracteres del documento encontrado:

In [ ]:
print(text[:500])

La información obtenida en formato `str` no resulta ser muy útil para un análisis. Como podemos ver aún contiene varias etiquetas propias del lenguaje **HTML** y no exclusivamente el contenido textual que es de nuestro interés.

Por ello, vamos a usar `beautifulsoup` para filtrar y extraer el contenido dentro de estos documentos. Comenzamos creando un objeto de tipo `BeautifulSoup` para que la librería interprete el contenido **HTML** de la página.

In [ ]:
soup = bs4.BeautifulSoup(text)

`BeautifulSoup` también genera un formato sobre el documento **HTML** para que sea más fácil de visualizar desde _Python_, veamos cómo se ve:

In [ ]:
print(soup)

El objeto `BeautifulSoup` estructura la información para ser extraída más fácilmente, por ejemplo, podemos extraer el título de la página:

In [ ]:
print(soup.title)

#### **3.1.4. Tipos de Filtros**
---

Con el objeto creado en `beautifulsoup` podemos filtrar elementos de interés, tenemos 3 formas de filtrar elementos:

* **Filtrado por etiqueta**: podemos filtrar todos los elementos del documento **HTML** que tengan una etiqueta específica (por ejemplo, todos los párrafos). Esto se consigue usando el método `find_all` al especificar una etiqueta. Veamos un ejemplo donde filtramos todos los headers de tipo `<h1>` (títulos de primer nivel).

In [ ]:
h1_vals = soup.find_all("h1")
print(h1_vals)

También podemos extraer todos los párrafos `<p>`:

In [ ]:
p_vals = soup.find_all("p")
print(p_vals)

Los resultados de esto se guardan como un objeto de tipo `Tag` y pueden ser usados para definir nuevos filtros y operaciones.

In [ ]:
print(type(p_vals[1]))
p_vals[1].text

Por ejemplo, del siguiente párrafo:

In [ ]:
par = p_vals[1]
print(par)

Si deseamos encontrar todos los hipervínculos `<a>` contenidos únicamente en este párrafo, podemos repetir el proceso anterior:

In [ ]:
links = par.find_all("a")
for link in links:
    print(link)

De la misma forma, el resultado será una nueva lista de `Tag` sobre el que podemos seguir trabajando:

In [ ]:
print(type(links[0]))

* **Filtrado por id**: en algunas oportunidades un elemento específico dentro de un **HTML** puede estar asociado a un identificador único, esto se ve de la siguiente forma dentro de cualquier etiqueta:

```html
<etiqueta id="valor"> Contenido </etiqueta>
```

Por ejemplo, dentro de la página estática que estamos analizando, hay una división `<div>` que contiene la tabla de contenido bajo el id `vector-toc`. Para filtrar estos elementos, también podemos usar el método `find_all` para encontrar todas las coincidencias. No obstante, como sabemos que los id son únicos podemos usar el método `find` para encontrar únicamente la primer coincidencia.

A diferencia del filtrado por etiquetas, ahora debemos usar el parámetro `id`. Veamos un ejemplo:

In [ ]:
toc = soup.find(id="vector-toc")
print(toc)

Como puede ver, encontramos el elemento `<div>` que corresponde a la tabla de contenido cuyo `id` es igual a `"vector-toc"` correspondiente a la tabla de contenido de la página web.

* **Filtrado por CSS o clase**: normalmente a los documentos **HTML** se les aplica un estilo por medio del [lenguaje CSS](https://www.w3schools.com/css/). Una etiqueta puede llevar un estilo específico como `class` dentro de su etiqueta, así:

```html
<etiqueta class="valor"> Contenido </etiqueta>
```

A través de la opción _Inspeccionar_ que está incluida en los navegadores web, se puede observar la estructura del archivo **HTML** de la página y el **CSS** asociado a la clase que está siendo utilizada por la etiqueta seleccionada. Esta opción generalmente se accede directamente pulsando la tecla `F12` mientras está en una página web o haciendo clic derecho sobre la página y seleccionando la opción `Inspeccionar`. Una vez seleccionada la opción, se abre una ventana dividida en dos: la parte superior correspondiente al código **HTML** y la parte inferior el **CSS** asociado.

A continuación se presenta un video con un ejemplo. Sin embargo, se recomienda que usted realice la inspección del código con alguna página web de su interés.

In [ ]:
#@markdown ##**Ejecute esta celda para ver el video.**
from IPython.display import IFrame
IFrame(
        src="https://drive.google.com/file/d/1Jbt1qgv8cQkMGAUIUCygF05Ba54-Ssva/preview",
        width="768px",
        height="432px"
        )

Así mismo, es posible filtrar con respecto a la clase de un elemento, con las funciones `find` y `find_all` que vimos anteriormente. La única diferencia es que se debe especificar el parámetro `class_`.

En el caso de la página que estamos manipulando, sabemos que contiene la clase `quote` para los elementos que se ven como citas, veamos un ejemplo del filtrado de las citas:

In [ ]:
quotes = soup.find_all(class_="quote")
for quote in quotes:
    print(quote)

#### **3.1.5. Extracción de Información**
---

Podemos extraer el contenido textual de cualquier elemento dentro de `BeautifulSoup`, esto se consigue con el método `get_text`.

Por ejemplo, el texto de la página web completa sería:

In [ ]:
full_text = soup.get_text()
print(full_text)

También es posible extraer el texto del resultado de un filtro. Por ejemplo, el texto de la tabla de contenido:

In [ ]:
toc = soup.find(id="vector-toc")
toc_text = toc.get_text()
print(toc_text)

Por último, veamos cómo extraer el texto de alguno de los párrafos (en este caso el párrafo `1`):

In [ ]:
p_vals = soup.find_all("p")
p_text = p_vals[1].get_text()
print(p_text)

### **3.2. Sitios Dinámicos**
---

Actualmente, muchas de las páginas web no tienen todo su contenido únicamente en un archivo **HTML**. De hecho, mucho del contenido textual de las páginas web modernas es generado desde código y se usan interacciones mediante el lenguaje de programación `JavaScript`.

Esto representa un inconveniente al momento de extraer la información únicamente con `requests` y `BeautifulSoup` ya que normalmente se requiere interactividad (presionar botones, llenar formularios, entre otros) para que el sitio web genere de forma dinámica el contenido.

Por este motivo, debemos recurrir a técnicas un poco más avanzadas para la extracción de información en estos casos.

#### **3.2.1. Instalación**
---

Para extraer información de páginas web dinámicas desde _Python_ se suele usar la herramienta `selenium`, la cual nos permite interactuar con un sitio web programáticamente como si lo estuviéramos haciendo desde nuestro propio navegador web.

<img src="https://drive.google.com/uc?export=view&id=1koHszh-qzeRy5YuY2eWmrMX4Diwtc8r9" width="50%">

Para usar `selenium` debemos instalar un driver de un navegador web. En este caso usaremos `chromium` para Google Chrome, comenzamos instalándolo dentro del entorno de Google Colab:

In [ ]:
!wget https://raw.githubusercontent.com/mindlab-unal/mlds4-datasets/main/u1/install_chromedriver.sh -O install_chromedriver.sh
!chmod +x install_chromedriver.sh && ./install_chromedriver.sh
!apt update
!apt install chromium-browser
!apt install chromium chromium-driver

Ahora, podemos instalar `selenium`:

In [ ]:
!pip install selenium

> **NOTA**: Después de la instalación, es posible que Google Colaboratory le pida reiniciar el entorno de ejecución (*RUNTIME*). Puede hacerlo, haciendo clic en el botón `RESTART RUNTIME` antes de continuar.

Importamos los módulos necesarios de `selenium` para **web scraping**:

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import bs4

#### **3.2.2. Configuración**
---

Como con `selenium` estamos usando todo un navegador web por detrás, debemos configurar algunos elementos del mismo. Esto se realiza con la clase `ChromeOptions` para crear un driver de tipo `Chrome` como se muestra a continuación:

In [ ]:
service = Service(executable_path=r'/usr/bin/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
driver = webdriver.Chrome(service=service, options=chrome_options)

Estas opciones significan:

* `headless`: no se crea ninguna ventana del navegador (como estamos trabajando en Google Colab, no tenemos acceso a ningún escritorio para ver el navegador.
* `no-sandbox`: evita que el driver se aisle para permitir interacción directa.

Note que creamos un objeto de tipo `WebDriver` sobre el que realizaremos todas las operaciones:

In [ ]:
print(type(driver))

#### **3.2.3. Carga de Datos**
---

En este caso, vamos a cargar la [página oficial de _Python_](https://www.python.org/).

Para acceder a esta página, usamos el método `get` del driver, usamos un `sleep` de 5 segundos para esperar que la página cargue (recuerde que prácticamente estamos usando un navegador).

In [ ]:
driver.get("https://www.python.org")
time.sleep(5)

Podemos tomar un pantallazo de lo que hay actualmente en el navegador con el método `get_screenshot_as_file` para crear una imagen:

In [ ]:
driver.get_screenshot_as_file("screen.png")

Veamos la imagen:

In [ ]:
img = mpimg.imread("screen.png")
fig, ax = plt.subplots(figsize=(10, 7))
ax.imshow(img)
ax.axis("off");

#### **3.2.4. Tipos de Filtros**
---

Desde `selenium` podemos seleccionar un elemento de la página e interactuar con el mismo. Para ello, usamos el método `find_element` para encontrar la primera coincidencia o el método `find_elements` para encontrar todos los valores que coincidan con el filtro. Los distintos filtros los encontramos en el módulo `By` de `selenium`.

Veamos un ejemplo donde seleccionamos la barra de búsqueda que está bajo el nombre `q`:

In [ ]:
search_bar = driver.find_element(By.NAME, "q")
print(search_bar)

También podemos encontrar la barra de búsqueda por id:

In [ ]:
search_bar2 = driver.find_element(By.ID, "id-search-field")
print(search_bar)

Los filtros sobre elementos de páginas en `selenium` incluyen:

* `By.NAME`: nombre de la etiqueta.

```html
<etiqueta name="valor"> Contenido </etiqueta>
```

* `By.ID`: identificador del elemento.

```html
<etiqueta id="valor"> Contenido </etiqueta>
```

* `By.CLASS_NAME`: nombre de la clase CSS.

```html
<etiqueta class="valor"> Contenido </etiqueta>
```

* `By.XPATH`: usando un `xpath` de elemento, puede revisar más información sobre xpath en [este enlace](https://www.w3schools.com/xml/xpath_intro.asp).
* `By.LINK_TEXT`: usando el valor de `href` del elemento.

```html
<a href="https://link.com"> Contenido </a>
```

* `By.TAG_NAME`: usando un tipo de tag especifico.

Adicional a esto, `selenium` nos permite interactuar con los elementos como si estuviéramos usando un puntero del *mouse* o ingresando texto. Por ejemplo, vamos a seleccionar la barra de búsqueda de la página y a insertar lo que deseamos buscar:

In [ ]:
search_bar = driver.find_element(By.NAME, "q")
search_bar.send_keys("regex")

El argumento `q` pasado a la función `find_element` proviene del nombre que tiene el atributo `name` en el tag `input` del archivo `html` de la página de *Python*. Este atributo especifica la denominación del elemento insertado en la barra de búsqueda. Esto se puede observar desde el navegador haciendo clic derecho sobre la barra y seleccionando la opción de *Inspeccionar* mencionada anteriormente, como se puede ver en el siguiente video:

In [ ]:
#@markdown ##**Ejecute esta celda para ver el video.**
from IPython.display import IFrame
IFrame(
        src="https://drive.google.com/file/d/1gCo4o1IbPBJQOlyQFN1SEdi2V1TGuS1M/preview",
        width="768px",
        height="432px"
        )

Podemos tomar un nuevo pantallazo para validar que efectivamente se insertó el texto `"regexp"` en la barra de búsqueda:

In [ ]:
driver.get_screenshot_as_file("screen.png")
img = mpimg.imread("screen.png")
fig, ax = plt.subplots(figsize=(10, 7))
ax.imshow(img)
ax.axis("off");

Ahora, vamos a seleccionar el botón asociado al botón de búsqueda:

In [ ]:
button = driver.find_element(By.NAME, "submit")
print(button)

Y hacemos clic sobre el botón con el método `click`:

In [ ]:
button.click()

#### **3.2.5. Extracción de Información**
---

Con `selenium` podemos obtener el código **HTML** de la página en su estado actual, accediendo al atributo `page_source` del driver:

In [ ]:
html = driver.page_source

Este valor lo podemos manipular directamente con `beautifulsoup` para extraer la información directamente:

In [ ]:
soup = bs4.BeautifulSoup(html)

Extraemos el texto de la página:

In [ ]:
print(soup.get_text())

## **Recursos Adicionales**
---

Los siguientes enlaces corresponden a sitios donde encontrará información muy útil para profundizar en los temas vistos en este notebook:

* [PyMuPDF documentation](https://pymupdf.readthedocs.io/en/latest/).
* [Tesseract user manual](https://tesseract-ocr.github.io/tessdoc/).
* [BeautifulSoup documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).
* [The Selenium Browser Automation Project](https://www.selenium.dev/documentation/).
* _Origen de los íconos_
    - Flaticon. Photo free icon [PNG]. https://www.flaticon.com/free-icon/photo_3771416
    - Flaticon. Notes free icon [PNG]. https://www.flaticon.com/free-icon/notes_3715004
    - Flaticon. Pdf File free icon [PNG]. https://www.flaticon.com/free-icon/pdf_4726010
    - Flaticon. Multifunction Printer free icon [PNG]. https://www.flaticon.com/free-icon/multifunction-printer_1547966
    - Flaticon. Title free icon [PNG]. https://www.flaticon.com/free-icon/title_2800015
    - Flaticon. Browser free icon [PNG]. https://www.flaticon.com/free-icon/browser_1183645
    - Flaticon. Csv free icon [PNG]. https://www.flaticon.com/free-icon/csv_202302
    - Daedalus. BeautifulSoup [PNG]. https://daedalus-ldv.de/wp-content/uploads/2022/02/bs-1024x440.png
    - Diego Molina. Selenium [PNG]. https://commons.wikimedia.org/wiki/File:Selenium_Logo.png

## **Créditos**

* **Profesor:** [Felipe Restrepo Calle](https://dis.unal.edu.co/~ferestrepoca/)
* **Asistentes docentes:**
    - [Juan Sebastián Lara Ramírez](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/).
* **Diseño de imágenes:**
    - [Rosa Alejandra Superlano Esquibel](mailto:rsuperlano@unal.edu.co).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*